# Statistical Analysis 

In [6]:
#Imports
import pandas as pd
from sklearn.decomposition import IncrementalPCA
from matplotlib.pyplot import scatter
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import itertools
import pycountry_convert as pc
from scipy.stats import mannwhitneyu
from statistics import median
import os
%matplotlib inline

In [2]:
path = '/home/karim/Downloads/Newmerged_data_2020.csv.gzip'  #Modify

In [7]:
dir_path = '../../'
filename = 'sample.csv'
sample = pd.read_csv(os.path.join(dir_path, filename))
sample.shape

(49235, 33)

In [8]:
sample = pd.concat(samples)
sample.drop_duplicates(inplace=True)
sample.reset_index()
sample.shape
sample.head(10)

NameError: name 'samples' is not defined

In [9]:
selected_feats = ['approx_word_count', 'token_count', 'adj_per_word', 'verb_per_word', 'base_ratio', 'pres_ratio', 'past_ratio', 'pronoun_per_word', 'sentiment']
speaker_feats = ['date_of_birth', 'nationality', 'gender', 'occupation', 'academic_degree', 'religion']

## Gender

In [10]:
mean_boxplots(sample,plot_by='gender',features=selected_feats)

NameError: name 'mean_boxplots' is not defined

## Occupation

In [ ]:
mean_boxplots(sample,plot_by='occupation',features=selected_feats)

## Academic Degree

In [ ]:
mean_boxplots(sample,plot_by='academic_degree',features=selected_feats)

## Religion

In [ ]:
mean_boxplots(sample,plot_by='religion',features=selected_feats)

## Generation

In [ ]:
def GroupBirthDates(sample):
    birth = sample.copy()
    birth["date_of_birth"] = birth["date_of_birth"].astype(int)
    birth["date_of_birth"] = birth["date_of_birth"].apply(lambda x: '30s' if x < 1940 else '50s' if x < 1960 else '70s' if x < 1980  else '90s' if x < 2000 else '00s' if x < 2020 else x)
    return birth

In [ ]:
birth = GroupBirthDates(sample)
mean_boxplots(birth,plot_by='date_of_birth',features=selected_feats, order=["30s","50s","70s","90s","00s"])

## Continent

In [ ]:
def GroupNationalities(sample):
    nat = sample.copy()
    for idx,i in enumerate(nat["nationality"]):
        try:
            nat["nationality"].iloc[idx] = pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(i, cn_name_format="default"))
        except:
            nat["nationality"].iloc[idx] = None
    return nat

In [ ]:
countries = GroupNationalities(sample)
mean_boxplots(countries,plot_by='nationality',features=selected_feats)

## Test the significance of the observed differences

In [11]:
def TestSignificance(df,lexical_features,speaker_features,tresh=0.05, verbose=False):
    Grid_max = np.zeros(shape=(len(lexical_features),len(speaker_features)))
    Grid_mean = np.zeros(shape=(len(lexical_features),len(speaker_features)))
    for idx_i,i in enumerate(speaker_features):                                      #Iterate over
        for idx_j,j in enumerate(lexical_features):                                  #Iterate over
            unique = df[i].unique()                  #Iterate over the different attributes
            unique = unique[unique != np.array(None)]
            unique = np.array(unique, dtype=str)
            unique = unique[unique != 'nan']
            unique = unique[unique != 'Other']
            pairs = list(itertools.combinations(unique,2))
            min_p = 1
            res = []
            for k in pairs:
                try:
                    N, p = mannwhitneyu(df[df[i]==k[0]][j], df[df[i]==k[1]][j])
                    if p < min_p:
                        min_p = p
                        min_x = k[0]
                        min_y = k[1]
                        param = j
                        speak = i
                        Grid_max[idx_j,idx_i] = 1/min_p
                    res.append(p)   
                except:
                    print('Test failed for {},{}'.format(i,j))
            if verbose:
                print('The maximum statistically significant difference with {} in {} is between {} and {}'.format(j,i,k[0],k[1]))
                print('The p-value is {}'.format(min_p))
            Grid_mean[idx_j,idx_i] = 1/median(res)
    return Grid_mean, Grid_max, lexical_features, speaker_features

In [12]:
Test = sample.copy()
Test["date_of_birth"] = birth["date_of_birth"]
Test["nationality"] = countries["nationality"]
#selected_feats = ['sentence_count', 'approx_word_count','adj_per_word','verb_per_word', 'base_ratio', 'pres_ratio',
#       'past_ratio', 'pronoun_per_word', 'self_ratio', 'union_ratio',
#       'other_ratio', 'sentiment']
selected_feats = ['sentence_count', '._per_sentence', ',_per_sentence', '!_per_sentence',
       '?_per_sentence', ':_per_sentence', ';_per_sentence', 'sign_per_token',
       'punctuation_per_sentence', 'approx_word_count', 'token_count',
       'adj_per_word', 'ordinal_ratio', 'comparative_ratio',
       'superlative_ratio', 'verb_per_word', 'base_ratio', 'pres_ratio',
       'past_ratio', 'pronoun_per_word', 'self_ratio', 'union_ratio',
       'other_ratio', 'sentiment']
speaker_feats = ['date_of_birth','nationality', 'gender', 'occupation', 'academic_degree', 'religion']
Grid_mean, Grid_max, lexical_features, speaker_features = TestSignificance(Test,selected_feats,speaker_feats,verbose=True)

NameError: name 'birth' is not defined

In [ ]:
df = pd.DataFrame(data=np.log(Grid_max), index=selected_feats, columns=speaker_feats)
fig, ax = plt.subplots(figsize=(20,10))         # Sample figsize in inches
plt.title("Heatmap of Median Significancy Score per Lexical and Speaker Feature Combination")
sns.heatmap(df, ax=ax)
plt.show()

In [ ]:
df = pd.DataFrame(data=np.log(Grid_mean), index=selected_feats, columns=speaker_feats)
fig, ax = plt.subplots(figsize=(20,10))         # Sample figsize in inches
sns.heatmap(df, ax=ax)
plt.title("Heatmap of Median Significancy Score per Lexical and Speaker Feature Combination")
plt.show()